In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import norm
import scipy.stats as stats
import math
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.writer.excel import save_virtual_workbook
from openpyxl.styles import Border, Side, Font
from datetime import datetime
from openpyxl.utils import get_column_letter

#### PNL

In [2]:
pnl_df = pd.read_csv('/Users/zhengyaojin/Desktop/Bluestar_PnL.csv')

In [3]:
pnl_clean = pnl_df.drop(columns=['category', 'feed', 'file', 'client', 'subset'])
pnl_clean['account_code'] = pnl_clean['account_code'].fillna('').apply(lambda x: str(int(x)) if x != '' else '')
#pnl_clean.head()

In [4]:
# Separate the df into four based on 'account_type'
sales_df = pnl_clean[pnl_clean['account_type'] == 'Sales']
direct_costs_df = pnl_clean[pnl_clean['account_type'] == 'Direct Costs']
other_income_df = pnl_clean[pnl_clean['account_type'] == 'Other Income']
expense_df = pnl_clean[pnl_clean['account_type'] == 'Expense']

sales_df_sorted = sales_df.sort_values(by='account')
direct_costs_df_sorted = direct_costs_df.sort_values(by='account')
other_income_df_sorted = other_income_df.sort_values(by='account')
expense_df_sorted = expense_df.sort_values(by='account')

In [5]:
# Rename 'account_type' and 'account' columns for sales_df_sorted, which will be the header for P&L spreadsheet
sales_df_sorted.rename(columns={'account_type': 'Account Type', 'account_code':'Account Code', 'account': 'Account'}, inplace=True)

# Rename date columns 
date_columns = sales_df_sorted.columns[3:]  
formatted_dates = [col.replace('_', ' ').title() for col in date_columns]
sales_df_sorted.columns = ['Account Type', 'Account Code', 'Account'] + formatted_dates

# Determine Actual or Budget
# Current month and year
current_month_year = datetime.now().strftime("%B %Y")

# Classify date as 'Actual' or 'Budget'
def classify_column(column_name):
    try:
        column_date = datetime.strptime(column_name, "%B %Y")
        current_date = datetime.strptime(current_month_year, "%B %Y")
        if column_date <= current_date:
            return 'Actual'
        else:
            return 'Budget'
    except ValueError:
        return ''

classification_list_sales = [''] * 3 + [classify_column(col) for col in sales_df_sorted.columns[3:]]
#print(classification_list_sales)

In [6]:
# This part will be deleted. Just to store hard-coded EBITDA number for now
# List of EBITDA values
ebitda_values = [
    445277, 90700, 10865, 156252, 401139, 368620, -184037, -47912, -151353, 
    -449466, -235587, -98650, 114116, 1413835, 456249, 371543, 494301, 
    605746, 601654, 147717, 182981, 489071, 136092, 467828, 573260
]

#### Excel Setup - PnL

In [7]:
# Create a new Excel workbook and a new sheet
wb = Workbook()
ws_1 = wb.active
ws_1.title = "P&L"

In [8]:
# Insert the classification list
for col_idx, value in enumerate(classification_list_sales, start=1):
    ws_1.cell(row=1, column=col_idx, value=value)

# # Append the data from sales_df_sorted to ws_1 row 2 and bold the Header
for row in dataframe_to_rows(sales_df_sorted, index=False, header=True):
    ws_1.append(row)
    
# Bold row 2
for cell in ws_1[2]:
    cell.font = Font(bold=True)

# Some Formatting
# Freeze the headers for ws_1
ws_1.freeze_panes = 'D3'

# Add a vertical border between columns C and D
for row in range(1, ws_1.max_row + 1):
    ws_1[f'C{row}'].border = Border(right=Side(style='thin'))

# Add a horizontal border only between rows 2 and 3
for col in ws_1.iter_cols(min_col=1, max_col=ws_1.max_column):
    ws_1[col[1].coordinate].border = Border(bottom=Side(style='thin'))
    
sales_rows = sales_df_sorted.shape[0] + 2
for cell in ws_1[sales_rows]:
    cell.border = Border(bottom=Side(style='thin'))

# Create a new row after the last row
total_sales_row = sales_rows + 1

# Add "Total Sales" to column A in the new row and bold it
ws_1[f'A{total_sales_row}'] = 'Total Sales'
for cell in ws_1[total_sales_row]:
    cell.font = Font(bold=True)

# Add the SUM formula from column D onwards
col_letter = 'D'
for col in range(4, ws_1.max_column + 1):  
    col_letter = ws_1.cell(row=1, column=col).column_letter  
    ws_1[f'{col_letter}{total_sales_row}'] = f'=SUM({col_letter}3:{col_letter}{sales_rows})'

# Insert an empty row after the "Total Sales" row
ws_1.append([])

In [9]:
# Append the data from direct_costs_df_sorted to ws_1, excluding the header
for row in dataframe_to_rows(direct_costs_df_sorted, index=False, header=False):
    ws_1.append(row)

row_count_dc = ws_1.max_row

# Add a border beneath Direct Cost
for cell in ws_1[row_count_dc]:
    cell.border = Border(bottom=Side(style='thin'))

# Create a new row after the last row
total_dc_row = row_count_dc + 1

# Add "Total Direct Costs" to column A in the new row and bold it
ws_1[f'A{total_dc_row}'] = 'Total Direct Costs'
for cell in ws_1[total_dc_row]:
    cell.font = Font(bold=True)

# Add the SUM formula from column C onwards
for col in range(4, ws_1.max_column + 1):  
    col_letter = ws_1.cell(row=1, column=col).column_letter  
    ws_1[f'{col_letter}{total_dc_row}'] = f'=SUM({col_letter}{row_count_dc - len(direct_costs_df_sorted) + 1}:{col_letter}{row_count_dc})'

# Insert an empty row after the "Total Direct Costs" row
cur_row = ws_1.max_row

for cell in ws_1[cur_row + 1]:
    cell.border = Border(bottom=Side(style='thin'))

gm_row = cur_row + 2

# Add "Gross Margin" to column A in the new row and bold it
ws_1[f'A{gm_row}'] = 'Gross Margin'
for cell in ws_1[gm_row]:
    cell.font = Font(bold=True)

# Add the formula to sum up "Total Sales" and "Total Direct Costs" from the respective rows
for col in range(4, ws_1.max_column + 1): 
    col_letter = ws_1.cell(row=1, column=col).column_letter  
    ws_1[f'{col_letter}{gm_row}'] = f'={col_letter}{total_sales_row} + {col_letter}{total_dc_row}'

for cell in ws_1[gm_row]:
    cell.border = Border(bottom=Side(style='thin'))

gm_percent_row = gm_row + 1

# Add "GM%" to column A in the new row and bold it
ws_1[f'A{gm_percent_row}'] = 'GM%'
for cell in ws_1[gm_percent_row]:
    cell.font = Font(bold=True, italic=True)

# Add the formula to calculate GM% (Gross Margin / Total Sales)
for col in range(4, ws_1.max_column + 1): 
    col_letter = ws_1.cell(row=1, column=col).column_letter  
    ws_1[f'{col_letter}{gm_percent_row}'] = f'={col_letter}{gm_row} / {col_letter}{total_sales_row}'
    ws_1[f'{col_letter}{gm_percent_row}'].number_format = '0.00%'

# Insert an empty row after the "Total Sales" row
ws_1.append([])

In [10]:
# Append the data from other_income_df_sorted to ws_1, excluding the header
for row in dataframe_to_rows(other_income_df_sorted, index=False, header=False):
    ws_1.append(row)

row_count_oi = ws_1.max_row

# Add a border beneath Other Income
for cell in ws_1[row_count_oi]:
    cell.border = Border(bottom=Side(style='thin'))

total_oi_row = row_count_oi + 1

# Add "Total Other Income" to column A in the new row and bold it
ws_1[f'A{total_oi_row}'] = 'Total Other Income'
for cell in ws_1[total_oi_row]:
    cell.font = Font(bold=True)

# Add the formula to sum up Total Other Income
for col in range(4, ws_1.max_column + 1): 
    col_letter = ws_1.cell(row=1, column=col).column_letter  
    ws_1[f'{col_letter}{total_oi_row}'] = f'=SUM({col_letter}{row_count_oi - len(other_income_df_sorted) + 1}:{col_letter}{row_count_oi})'

# Insert an empty row after the "Total Other Income" row
ws_1.append([])

In [11]:
# Append the data from expense_df_sorted to ws_1, excluding the header
for row in dataframe_to_rows(expense_df_sorted, index=False, header=False):
    ws_1.append(row)

row_count_exps = ws_1.max_row

# Add a border beneath Expense
for cell in ws_1[row_count_exps]:
    cell.border = Border(bottom=Side(style='thin'))

total_exps_row = row_count_exps + 1

# Add "Total Expense" to column A in the new row and bold it
ws_1[f'A{total_exps_row}'] = 'Total Expense'
for cell in ws_1[total_exps_row]:
    cell.font = Font(bold=True)

# Add the formula to sum up Total Expense
for col in range(4, ws_1.max_column + 1): 
    col_letter = ws_1.cell(row=1, column=col).column_letter  
    ws_1[f'{col_letter}{total_exps_row}'] = f'=SUM({col_letter}{row_count_exps - len(expense_df_sorted) + 1}:{col_letter}{row_count_exps})'

In [12]:
cur_row_count = ws_1.max_row

opt_row_count = cur_row_count + 2

# Add a border above Operating Income
for cell in ws_1[cur_row_count + 1]:
    cell.border = Border(bottom=Side(style='thin'))

# Add a border beneath Operating Income
for cell in ws_1[opt_row_count]:
    cell.border = Border(bottom=Side(style='thin'))
    
# Add "Operating Income" to column A in the new row and bold it
ws_1[f'A{opt_row_count}'] = 'Operating Income'
for cell in ws_1[opt_row_count]:
    cell.font = Font(bold=True)

# Add the formula to sum up "Operating Income" from the respective rows
for col in range(4, ws_1.max_column + 1): 
    col_letter = ws_1.cell(row=1, column=col).column_letter  
    ws_1[f'{col_letter}{opt_row_count}'] = f'={col_letter}{gm_row} + {col_letter}{total_oi_row} + {col_letter}{total_exps_row}'

# Insert an empty row after the "Total Expense" row
ws_1.append([])

In [13]:
ebitda_row_count = opt_row_count + 2

# Add a border above EBITDA
for cell in ws_1[ebitda_row_count - 1]:
    cell.border = Border(bottom=Side(style='thin'))

# Add a double border beneath EBITDA
for cell in ws_1[ebitda_row_count]:
    cell.border = Border(bottom=Side(style='double'))

# Add "EBITDA" to column A in the new row and bald it
ws_1[f'A{ebitda_row_count}'] = 'EBITDA'
for cell in ws_1[ebitda_row_count]:
    cell.font = Font(bold=True)

# Add the formula to calculate "EBITDA" from the respective rows
# This part is now hard-coded number


#for col in range(3, ws_1.max_column + 1): 
#    col_letter = ws_1.cell(row=1, column=col).column_letter  
#    ws_1[f'{col_letter}{ebitda_row_count}'] = 

# Need to update with actual formula in the future 
for col_idx, value in enumerate(ebitda_values, start=4):
    ws_1.cell(row=ebitda_row_count, column=col_idx, value=value)

ebitada_percent_row = ebitda_row_count + 1

# Add "EBITDA %" to column A in the new row and bold it
ws_1[f'A{ebitada_percent_row}'] = 'EBITDA %'
for cell in ws_1[ebitada_percent_row]:
    cell.font = Font(bold=True, italic=True)
    
# Add the formula to calculate EBITDA % (EBITDA / Total Sales)
for col in range(4, ws_1.max_column + 1): 
    col_letter = ws_1.cell(row=1, column=col).column_letter  
    ws_1[f'{col_letter}{ebitada_percent_row}'] = f'={col_letter}{ebitda_row_count} / {col_letter}{total_sales_row}'
    ws_1[f'{col_letter}{ebitada_percent_row}'].number_format = '0.00%'


In [14]:
# Calculate rolling sum
def calculate_rolling_sum(year, beginning_date, end_date):
    header_row = 2  
    date_columns = {}
    
    # Find all date columns and map them with column indices
    for col in range(4, ws_1.max_column + 1):
        col_header = ws_1.cell(row=header_row, column=col).value
        try:
            date = datetime.strptime(col_header, "%B %Y")
            date_columns[date] = col
        except (ValueError, TypeError):
            continue

    # Convert beginning_date and end_date strings to datetime objects 
    fy_start_date = datetime(year - 1, datetime.strptime(beginning_date, "%B").month, 1)
    fy_end_date = datetime(year, datetime.strptime(end_date, "%B").month, 1)
    
    # Select columns within the fiscal year range
    fy_columns = {
        d: col
        for d, col in date_columns.items()
        if fy_start_date <= d <= fy_end_date
    }

    if fy_columns:
        # Append to the end
        rolling_sum_col_idx = ws_1.max_column + 1
        ws_1.insert_cols(rolling_sum_col_idx)
        
        # Insert the rolling sum column header
        new_header_cell = ws_1.cell(row=header_row, column=rolling_sum_col_idx)
        new_header_cell.value = f'FY{year} Total'
        
        # Set column width to the length of the header
        column_width = len(new_header_cell.value)
        ws_1.column_dimensions[get_column_letter(rolling_sum_col_idx)].width = column_width
        
        # Bold the entire column
        bold_font = Font(bold=True)
        new_header_cell.font = bold_font
        
        # Calculate rolling sum for each row and bold the entire column
        for row in range(3, ws_1.max_row + 1):  
            sum_formula_parts = [
                f"{get_column_letter(col_idx)}{row}"
                for col_idx in fy_columns.values()
            ]
            sum_formula = f"=SUM({','.join(sum_formula_parts)})"
            new_cell = ws_1.cell(row=row, column=rolling_sum_col_idx)
            new_cell.value = sum_formula
            new_cell.font = bold_font

# For this company, the fiscal year ranges from July to June            
calculate_rolling_sum(2024, 'July', 'June')

In [15]:
# Dynamically adjust column width
max_length_account = pnl_clean['account'].str.len().max()
max_length_account_type = pnl_clean['account_type'].str.len().max()
max_length_account_code = pnl_clean['account_code'].str.len().max()

ws_1.column_dimensions['A'].width = max_length_account_type 
ws_1.column_dimensions['B'].width = max_length_account_code + 5
ws_1.column_dimensions['C'].width = max_length_account 

# Set the font size to 9 for all cells in the worksheet ws_1
for row in ws_1.iter_rows():
    for cell in row:
        cell.font = Font(size=9, bold=cell.font.bold, italic=cell.font.italic)

# Apply comma format to all numeric columns 
for col in range(4, ws_1.max_column + 1):  
    for row in range(2, ws_1.max_row + 1):  
        cell = ws_1.cell(row=row, column=col)
        if cell.number_format != '0.00%':
            cell.number_format = '#,##0'  

#### Balance Sheet

In [16]:
bs_df = pd.read_csv('/Users/zhengyaojin/Desktop/Bluestar_BalanceSheet_v2.csv', dtype={'account_code': str})

In [17]:
bs_clean = bs_df.drop(columns = ['category', 'feed', 'file', 'client', 'subset'])

asset_df = bs_clean[bs_clean['account_type'].isin(['Bank', 'Current Asset', 'Non-current Asset', 'Fixed Asset'])]
liability_df = bs_clean[bs_clean['account_type'].isin(['Current Liability', 'Non-current Liability'])]
equity_df = bs_clean[bs_clean['account_type'].isin(['Equity'])]

# Rename 'account_type' and 'account' columns for asset_df, which will be the header for BS spreadsheet
asset_df.rename(columns={'account_type': 'Account Type', 'account_code': 'Account Code', 'account': 'Account'}, inplace=True)

# Rename date columns 
date_columns_asset = asset_df.columns[3:]  
formatted_dates_asset = [col.replace('_', ' ').title() for col in date_columns_asset]
asset_df.columns = ['Account Type', 'Account Code', 'Account'] + formatted_dates_asset

/var/folders/49/pxgfdjk93j37m84dbmvq4ndh0000gn/T/ipykernel_99718/3029256611.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_df.rename(columns={'account_type': 'Account Type', 'account_code': 'Account Code', 'account': 'Account'}, inplace=True)


In [18]:
# Determine Actual or Budget
# Current month and year
current_month_year = datetime.now().strftime("%B %Y")

# Classify date as 'Actual' or 'Budget'
def classify_column(column_name):
    try:
        column_date = datetime.strptime(column_name, "%B %Y")
        current_date = datetime.strptime(current_month_year, "%B %Y")
        if column_date <= current_date:
            return 'Actual'
        else:
            return 'Budget'
    except ValueError:
        return ''

classification_list = [''] * 3 + [classify_column(col) for col in asset_df.columns[3:]]

#### Excel Setup - BS

In [19]:
ws_2 = wb.create_sheet(title="BS")

In [20]:
# # Leave a blank row for listing 'Actual' later on
# ws_2.append([])

# Insert the classification list
for col_idx, value in enumerate(classification_list, start=1):
    ws_2.cell(row=1, column=col_idx, value=value)

# Append asset_df
for row in dataframe_to_rows(asset_df, index=False, header=True):
    ws_2.append(row)

# Some Formatting
# Freeze the headers
ws_2.freeze_panes = 'D3'

# Add a vertical border between columns C and D
for row in range(1, ws_2.max_row + 1):
    ws_2[f'C{row}'].border = Border(right=Side(style='thin'))

# Add a horizontal border only between rows 2 and 3
for col in ws_2.iter_cols(min_col=1, max_col=ws_2.max_column):
    ws_2[col[1].coordinate].border = Border(bottom=Side(style='thin'))


for cell in ws_2[2]:
    cell.font = Font(bold=True)

row_count = ws_2.max_row
total_asset_row = row_count + 2

# Add a border above Total Asset
for cell in ws_2[total_asset_row - 1]:
    cell.border = Border(bottom=Side(style='thin'))

# Add a border beneath Total Asset
for cell in ws_2[total_asset_row]:
    cell.border = Border(bottom=Side(style='thin'))

# Add "Total Assets" to column A in the new row and bold it
ws_2[f'A{total_asset_row}'] = 'Total Assets'
for cell in ws_2[total_asset_row]:
    cell.font = Font(bold=True)

# Add the formula to sum up "Total Assets"
for col in range(4, ws_2.max_column + 1):  
    col_letter = ws_2.cell(row=1, column=col).column_letter
    ws_2[f'{col_letter}{total_asset_row}'] = f'=SUM({col_letter}{row_count - len(asset_df) + 1}:{col_letter}{row_count})'
        
# Insert an empty row after the "Total Expense" row
ws_2.append([])

In [21]:
# Append liability_df
for row in dataframe_to_rows(liability_df, index=False, header=False):
    ws_2.append(row)

row_count_liability = ws_2.max_row
total_liability_row = row_count_liability + 2

# Add a border above Total Liabilities
for cell in ws_2[total_liability_row - 1]:
    cell.border = Border(bottom=Side(style='thin'))

# Add a border beneath Total Liabilities
for cell in ws_2[total_liability_row]:
    cell.border = Border(bottom=Side(style='thin'))

# Add "Total Liabilities" to column A in the new row and bold it
ws_2[f'A{total_liability_row}'] = 'Total Liabilities'
for cell in ws_2[total_liability_row]:
    cell.font = Font(bold=True)

# Add the formula to sum up "Total Liabilities"
for col in range(4, ws_2.max_column + 1):  
    col_letter = ws_2.cell(row=1, column=col).column_letter
    ws_2[f'{col_letter}{total_liability_row}'] = f'=SUM({col_letter}{row_count_liability - len(liability_df) + 1}:{col_letter}{row_count_liability})'
        
# Insert an empty row after the "Total Liabilities" row
ws_2.append([])

In [22]:
# Append equity_df
for row in dataframe_to_rows(equity_df, index=False, header=False):
    ws_2.append(row)

row_count_equity = ws_2.max_row
total_equity_row = row_count_equity + 2

# Add a border above Total Equity
for cell in ws_2[total_equity_row - 1]:
    cell.border = Border(bottom=Side(style='thin'))

# Add a border beneath Total Equity
for cell in ws_2[total_equity_row]:
    cell.border = Border(bottom=Side(style='thin'))

# Add "Total Equity" to column A in the new row and bold it
ws_2[f'A{total_equity_row}'] = 'Total Equity'
for cell in ws_2[total_equity_row]:
    cell.font = Font(bold=True)

# Add the formula to sum up "Total Equity"
for col in range(4, ws_2.max_column + 1):  
    col_letter = ws_2.cell(row=1, column=col).column_letter
    ws_2[f'{col_letter}{total_equity_row}'] = f'=SUM({col_letter}{row_count_equity - len(equity_df) + 1}:{col_letter}{row_count_equity})'
        

In [23]:
# Some more formatting
# Dynamically adjust column width
max_length_account_bs = bs_clean['account'].str.len().max()
max_length_account_type_bs = bs_clean['account_type'].str.len().max()
max_length_account_code_bs = bs_clean['account_code'].str.len().max()

ws_2.column_dimensions['A'].width = max_length_account_type_bs
ws_2.column_dimensions['B'].width = max_length_account_code_bs + 5
ws_2.column_dimensions['C'].width = max_length_account_bs

# Set the font size to 9 for all cells in the worksheet ws_2
for row in ws_2.iter_rows():
    for cell in row:
        cell.font = Font(size=9, bold=cell.font.bold, italic=cell.font.italic)

# Apply comma format to all numeric columns 
for col in range(4, ws_2.max_column + 1):  
    for row in range(3, ws_2.max_row + 1):  
        cell = ws_2.cell(row=row, column=col)
        if cell.number_format != '0.00%':
            cell.number_format = '#,##0'

#### CF - Excel Setup

In [24]:
ws_3 = wb.create_sheet(title="CF")

In [25]:
# Need to substitue these with actual formulas later
# Hard-coded numbers
# Change in AR
change_in_ar = [349919, -13324161, -1399573, -399489, 1951098, 822518, -416401, -312316, -853755, -735571, 1234560]

# Change in Accrued Payroll
change_in_accrued_payroll = [-255345, 151925, 3, 9733, -7932, 138973, 115188, 7991, -262784, 267136, 15024]

# Interest
interest = [-125000, -144883, -127205, -121461, -119024, -132453, -120764, -117777, -131627, -124217, -84579]

# GST
gst = [806582, -887397, 490155, -273579, 1301463, -699267, -737958, 0, 1149943, -3733, 249784]

# Change in Inventory
change_in_inventory = [122517, -20755, -7897, 36647, -8451, 9722, 7342, -64795, -27706, -22332, -18981]

# Change in A/P and Accrued Liabilities
change_in_ap_and_accrued_liabilities = [-1550383, -276295, 4541, 243661, 148890, -60913, 0, 12887571, 1904475, 140043, 318384]

In [26]:
# Copy the header 
for col_idx, cell in enumerate(ws_2[2], start=1):  
    ws_3.cell(row=1, column=col_idx, value=cell.value)

# Bold the first row
for cell in ws_3[1]:
    cell.font = Font(bold=True)

# Get ebitda from Income Statement
# Set ebitda row in CF
ebitda_cf_row = ws_3.max_row + 1

ws_3[f'A{ebitda_cf_row}'] = 'EBITDA'

# EBITDA
for col in range(4, ws_1.max_column):  
    ws_3_col = col
    col_letter_ws1 = ws_1.cell(row=1, column=col).column_letter  
    col_letter_ws3 = ws_3.cell(row=1, column=ws_3_col).column_letter  
    ws_3[f'{col_letter_ws3}{ebitda_cf_row}'] = f"='{ws_1.title}'!{col_letter_ws1}{ebitda_row_count}"

# Change in AR
ar_cf_row = ebitda_cf_row + 2
ws_3[f'A{ar_cf_row}'] = 'Change in AR'

# Add numbers from column F onwards
start_col = 6 

for i, value in enumerate(change_in_ar):
    col_letter = get_column_letter(start_col + i)
    ws_3[f'{col_letter}{ar_cf_row}'] = value
    
# Change in AP
ap_cf_row = ar_cf_row + 1
ws_3[f'A{ap_cf_row}'] = 'Change in Accrued Payroll'

for i, value in enumerate(change_in_accrued_payroll):
    col_letter = get_column_letter(start_col + i)
    ws_3[f'{col_letter}{ap_cf_row}'] = value


# Interest
interest_cf_row = ap_cf_row + 1
ws_3[f'A{interest_cf_row}'] = 'Interest'

for i, value in enumerate(interest):
    col_letter = get_column_letter(start_col + i)
    ws_3[f'{col_letter}{interest_cf_row}'] = value
    

# GST
gst_cf_row = interest_cf_row + 1
ws_3[f'A{gst_cf_row}'] = 'GST'

for i, value in enumerate(gst):
    col_letter = get_column_letter(start_col + i)
    ws_3[f'{col_letter}{gst_cf_row}'] = value


# Change in inventory
inventory_cf_row = gst_cf_row + 1
ws_3[f'A{inventory_cf_row}'] = 'Change in Inventory'

for i, value in enumerate(change_in_inventory):
    col_letter = get_column_letter(start_col + i)
    ws_3[f'{col_letter}{inventory_cf_row}'] = value
    

# Change in A/P and accured liabilities
liabilities_cf_row = inventory_cf_row + 1
ws_3[f'A{liabilities_cf_row}'] = 'Change in A/P and Accured Liabilities'

for i, value in enumerate(change_in_ap_and_accrued_liabilities):
    col_letter = get_column_letter(start_col + i)
    ws_3[f'{col_letter}{liabilities_cf_row}'] = value
    
operating_cf_row = liabilities_cf_row + 2

# Add "Operating CF" to column A in the new row and bold it
ws_3[f'A{operating_cf_row}'] = 'Operating CF'
for cell in ws_3[operating_cf_row]:
    cell.font = Font(bold=True)

# Apply the border and bold to operating cash flow
for cell in ws_3[operating_cf_row]:
    cell.border = Border(top=Side(style='thin'), bottom=Side(style='thin'))
    cell.font = Font(bold=True)


# Sum the values from ebitda_cf_row to liabilities_cf_row
for col in range(4, ws_3.max_column + 1):
    col_letter = get_column_letter(col)
    sum_formula = f"=SUM({col_letter}{ebitda_cf_row}:{col_letter}{liabilities_cf_row})"
    ws_3[f'{col_letter}{operating_cf_row}'] = sum_formula


In [27]:
# Set the font size to 9 for all cells in ws_3
for row in ws_3.iter_rows():
    for cell in row:
        cell.font = Font(size=9, bold=cell.font.bold, italic=cell.font.italic)

# Apply comma format to all numeric columns 
for col in range(4, ws_3.max_column + 1):  
    for row in range(2, ws_3.max_row + 1):  
        cell = ws_3.cell(row=row, column=col)
        if cell.number_format != '0.00%':
            cell.number_format = '#,##0'

In [28]:
wb.save('PnL_BS_CF.xlsx')